## Oracle Definition ##

This implements the oracle $D |z\rangle |k\rangle = |z \oplus x_k \rangle |k\rangle$ used in the Grover
search algorithm.

In [ ]:
operation ApplyDatabaseOracle(
    markedElement : Int,
    markedQubit: Qubit,
    databaseRegister: Qubit[]
) : Unit is Adj + Ctl {
    ControlledOnInt(markedElement, X)(databaseRegister, markedQubit);

}

In [ ]:
ControlledOnInt?

## State Preparation ##

This implements an oracle $DU$ that prepares the start state
$DU |0\rangle|0\rangle = 1 / \sqrt{N} |1\rangle|\text{marked}\rangle + \sqrt{N - 1} / \sqrt{N} |0\rangle|\text{unmarked}\rangle$ where $N = 2^n$.

In [ ]:
open Microsoft.Quantum.Arrays;

operation PrepareDatabaseRegister(
    markedElement : Int,
    idxMarkedQubit: Int,
    startQubits: Qubit[]
) : Unit is Adj + Ctl {
    let flagQubit = startQubits[idxMarkedQubit];
    let databaseRegister = Exclude([idxMarkedQubit], startQubits);

    // Apply 𝑈.
    ApplyToEachCA(H, databaseRegister);

    // Apply 𝐷.
    ApplyDatabaseOracle(markedElement, flagQubit, databaseRegister);
}


Here, we wrap our state preparation in a *user-defined type* to indicate that it is a state preparation oracle.

In [ ]:
open Microsoft.Quantum.Oracles;

function GroverStatePrepOracle(markedElement : Int) : StateOracle {
    return StateOracle(PrepareDatabaseRegister(markedElement, _, _));
}

## Grover's Algorithm ##

In [ ]:
open Microsoft.Quantum.AmplitudeAmplification;

function GroverSearch(
    markedElement: Int,
    nIterations: Int,
    idxMarkedQubit: Int
) : (Qubit[] => Unit is Adj + Ctl) {

    return StandardAmplitudeAmplification(nIterations, GroverStatePrepOracle(markedElement), idxMarkedQubit);

}

In [ ]:
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

operation ApplyQuantumSearch() : (Result, Int) {

    let nIterations = 6;
    let nDatabaseQubits = 6;
    let markedElement = 3;

    use markedQubit = Qubit();
    use databaseRegister = Qubit[nDatabaseQubits];

    // Implement the quantum search algorithm.
    GroverSearch(markedElement, nIterations, 0)([markedQubit] + databaseRegister);

    // Measure the marked qubit. On success, this should be One.
    let resultSuccess = MResetZ(markedQubit);

    // Measure the state of the database register post-selected on
    // the state of the marked qubit.
    let resultElement = ForEach(MResetZ, databaseRegister);
    let numberElement = ResultArrayAsInt(resultElement);

    // Returns the measurement results of the algorithm.
    return (resultSuccess, numberElement);
    
}

In [ ]:
%simulate ApplyQuantumSearch

## Epilogue ##

In [ ]:
%version